In [1]:
## imports
import numpy as np
import cvxpy as cp
import pandas as pd
import matplotlib.pyplot as plt
import itertools
%run ccpp_light.ipynb
from IPython.display import clear_output
%matplotlib inline
%config InlineBackend.figure_format='retina'
clear_output()

### data

In [2]:
dic_results = {'UB':[],'LB':[],'N':[],'repID':[],'time':[],'certif':[],\
               'd':[],'tau':[],'p':[],'skip_':[]}

In [3]:
''' dimensions & hyper-parameters'''

np.random.seed(2308)

d_list = [int(3)]
skip_list = [False,True]
N = int(5e2)
tau_list =  [.05,.15,.25]
N_reps = int(5)
otilde = 1
o = 2
TL = 720
MT_cert = 120

''' commands '''

xi_inf_bound = 5
x_inf_bound = 10
granularity = 6
frac = 23/50

''' elements  '''
def rnd_pos(p):
    buf = np.zeros(p)
    id_pos = np.random.randint(0,p)
    buf[id_pos]= np.random.choice(np.linspace(-xi_inf_bound,xi_inf_bound,granularity))
    denom = np.random.randint(1,100,p-1)
    buf[np.setdiff1d(np.arange(p),[id_pos])] = xi_inf_bound*np.random.randint(-denom,denom+1,p-1)/denom
    return buf


# from INRIA 
def proj_simplex_(vec, R=1,tol=1e-3):
    """ 
    References
    ----------
    [1] Efficient Projections onto the .1-Ball for Learning in High Dimensions
        John Duchi, Shai Shalev-Shwartz, Yoram Singer, and Tushar Chandra.
        International Conference on Machine Learning (ICML 2008)
        http://www.cs.berkeley.edu/~jduchi/projects/DuchiSiShCh08.pdf
    """
    assert R > 0, "radius R must be strictly positive (%d <= 0)" % R
    d, = vec.shape  # will raise ValueError if vec is not 1-D
    if abs(vec.sum() - R)<=tol and np.alltrue(vec >= 0):
        return vec
    sorted_vec = np.sort(vec)[::-1]
    cssv = np.cumsum(sorted_vec)
    buf = np.nonzero(sorted_vec * np.arange(1, d+1) > (cssv - R))[0][-1]
    shift = float(cssv[buf] - R) / (1+buf)
    w = (vec - shift).clip(min=0)
    return w

## ball proximity based
def proj_ball_(x,xi_embd,nm_power,R,tol=5e-4):
    dxxi_embd = x-xi_embd
    abs_dxxi_embd = np.abs(dxxi_embd)
    if nm_power==1:
        if np.sum(abs_dxxi_embd)<=R+tol:
            return x
        return xi_embd + proj_simplex_(abs_dxxi_embd,R)*np.sign(dxxi_embd)
    elif nm_power==np.inf:
        if max(abs_dxxi_embd)<=R+tol:
            return x
        return xi_embd + np.minimum(R,np.maximum(-R,dxxi_embd))
    elif nm_power==2:
        buf = (np.sum(abs_dxxi_embd**2))**(1/2)
        if buf<=R+tol:
            return x
        return xi_embd + dxxi_embd*R/max(R,buf)
    else:
        return None

In [ ]:
for d in d_list:

    print('====> d = '+str(d))
    print(' ')

    p = d
    
    R_base = 2*d*xi_inf_bound*frac

    x_ref = np.random.choice([-xi_inf_bound,xi_inf_bound],d) + ( (np.arange(d))**2 )/d

    ## numpy set-up ##
    if o==2:
        def obj_val(x):
            return np.sum((x-x_ref)**2)
    else:
        def obj_val(x):
            return np.linalg.norm(x-x_ref,o)
            
    ## cvxpy set-up ##
    x_cvx = cp.Variable(d)
    xi_param = cp.Parameter(p)
    big_bound = max(np.max(np.abs(x_ref)),xi_inf_bound+R_base) # big enough so that min_x in X of F(x) s.t. x in R(s) is the same as min_x of F(x) s.t. x in R(s)
    X = [x_cvx>=-big_bound,x_cvx<=big_bound]
    
    if o==2:
        obj_ = cp.sum_squares(x_cvx-x_ref)
    else:
        obj_ = cp.norm(x_cvx-x_ref,o)

    def cstr_gen_cvxpy(xi_embd,nm_power=otilde,R=R_base):
        return cp.norm(x_cvx-xi_embd,nm_power)-R

    
    if otilde==1:
        
        if p==2:
            c_bound_cvxpy = [x_cvx[0]-xi_param[0]+x_cvx[1]-xi_param[1],\
                            -x_cvx[0]+xi_param[0]+x_cvx[1]-xi_param[1],\
                            x_cvx[0]-xi_param[0]-x_cvx[1]+xi_param[1],\
                            -x_cvx[0]+xi_param[0]-x_cvx[1]+xi_param[1]]
        elif p==3:
            c_bound_cvxpy = [x_cvx[0]-xi_param[0]+x_cvx[1]-xi_param[1]+x_cvx[2]-xi_param[2],\
                            -x_cvx[0]+xi_param[0]+x_cvx[1]-xi_param[1]+x_cvx[2]-xi_param[2],\
                            x_cvx[0]-xi_param[0]-x_cvx[1]+xi_param[1]+x_cvx[2]-xi_param[2],\
                            -x_cvx[0]+xi_param[0]-x_cvx[1]+xi_param[1]+x_cvx[2]-xi_param[2],\
                            x_cvx[0]-xi_param[0]+x_cvx[1]-xi_param[1]-x_cvx[2]+xi_param[2],\
                            -x_cvx[0]+xi_param[0]+x_cvx[1]-xi_param[1]-x_cvx[2]+xi_param[2],\
                            x_cvx[0]-xi_param[0]-x_cvx[1]+xi_param[1]-x_cvx[2]+xi_param[2],\
                            -x_cvx[0]+xi_param[0]-x_cvx[1]+xi_param[1]-x_cvx[2]+xi_param[2]]
    elif otilde==np.inf:
        
        c_bound_cvxpy = []
        for j in range(p):
            for signum in [-1,1]:
                c_bound_cvxpy += [signum*(x_cvx[j]-xi_param[j])]
            
    else:
        
        print('not yet ')

    ################

    def single_s_specialized(xi_s):
        x_s = proj_ball_(x_ref,xi_s,otilde,R_base)
        return obj_val(x_s),[x_s]


    for repID in range(N_reps):

        print(' ')
        print('rep #'+str(repID+1))
        print(' ')

        # xi data generation
        xi_list = np.array([rnd_pos(int(p)) for s in range(N)])
        pis = np.ones(N)/N

        for tau in tau_list:

                print('tau = '+str(tau))
                print(' ')

                for sk in skip_list:

                    print('skip_presolve: '+str(sk))


                    prob = Problem(obj_,X,tau,xi_list,pis,cstr_gen=cstr_gen_cvxpy,\
                                 c_bound=c_bound_cvxpy,rel_tol=1e-2,used_solver=cp.MOSEK,individual_nus_comp=single_s_specialized)
                    

                    stat,timing,_,_,_,_ = prob.solve(include_cuts=False,verbose_general=True,T_max=TL,skip_presolve=sk,\
                                      verbose_certify=True,verbose_prune=True,display=False,max_time_cert=MT_cert,max_rounds=1)
                    dic_results['UB'].append(prob.UB)
                    dic_results['LB'].append(prob.LB)
                    dic_results['N'].append(N)
                    dic_results['repID'].append(repID)
                    dic_results['time'].append(timing)
                    dic_results['certif'].append(stat)
                    dic_results['d'].append(d)
                    dic_results['tau'].append(tau)
                    dic_results['p'].append(p)
                    dic_results['skip_'].append(sk)
                    print(' ')

                    df = pd.DataFrame(data=dic_results)
                    df.to_csv('benchmark_final_p_'+str(p)+'_o_'+str(o)+'_otilde_'+str(otilde)+'.csv')
                

====> d = 3
 
 
rep #1
 
tau = 0.05
 
skip_presolve: False
 
pre-pass oracle...
LB = 0.0 | self.UB = 39.30062733590166 | pre-pruned: 11
 
 
empirical cstr (LB): 0.7180000000000002
current lower-bound: 5.576333165743179
 
 
empirical cstr (UB): 0.9540000000000004
current upper-bound: 37.06533832827507
 
 
trial @1 | remaining checks [%] -> 99.23
trial @41 | remaining checks [%] -> 89.23
trial @97 | remaining checks [%] -> 79.23
trial @152 | remaining checks [%] -> 69.23
trial @208 | remaining checks [%] -> 59.23
pruned! @224
trial @284 | remaining checks [%] -> 49.23
trial @325 | remaining checks [%] -> 39.23
pruned! @325
trial @362 | remaining checks [%] -> 29.23
trial @407 | remaining checks [%] -> 19.23
trial @460 | remaining checks [%] -> 9.230
 
after optimality scan: 2 indices were added to the pruned set
 
 
 /!\ pre_solve termination /!\ 
safe indices: 22 ; pruned indices: 13
best feasible solution:: 37.06533832827507
                                     CVXPY                   

(CVXPY) Mar 08 04:49:06 PM: 7106     7090     3651     67       4.8605893905e+01     3.4522815694e+01     28.97       85.4  
(CVXPY) Mar 08 04:49:09 PM: 7395     7379     3742     134      4.8605893905e+01     3.4522815694e+01     28.97       88.8  
(CVXPY) Mar 08 04:49:13 PM: 7683     7667     3858     157      4.8605893905e+01     3.4522815694e+01     28.97       92.3  
(CVXPY) Mar 08 04:49:16 PM: 7973     7957     3924     180      4.8605893905e+01     3.4522815694e+01     28.97       95.6  
(CVXPY) Mar 08 04:49:19 PM: 8260     8244     4047     203      4.8605893905e+01     3.4522815694e+01     28.97       99.0  
(CVXPY) Mar 08 04:49:23 PM: 8545     8529     4196     226      4.8605893905e+01     3.4522815694e+01     28.97       102.4 
(CVXPY) Mar 08 04:49:26 PM: 8836     8820     4287     223      4.8605893905e+01     3.4522815694e+01     28.97       105.7 
(CVXPY) Mar 08 04:49:29 PM: 9127     9111     4282     72       4.8605893905e+01     3.4522815694e+01     28.97       109.0 


(CVXPY) Mar 08 04:51:32 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Mar 08 04:51:33 PM: Applying reduction MOSEK
(CVXPY) Mar 08 04:51:34 PM: Finished problem compilation (took 2.604e+00 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Mar 08 04:51:34 PM: Invoking solver MOSEK  to obtain a solution.


(CVXPY) Mar 08 04:51:34 PM: Problem
(CVXPY) Mar 08 04:51:34 PM:   Name                   :                 
(CVXPY) Mar 08 04:51:34 PM:   Objective sense        : minimize        
(CVXPY) Mar 08 04:51:34 PM:   Type                   : CONIC (conic optimization problem)
(CVXPY) Mar 08 04:51:34 PM:   Constraints            : 4013            
(CVXPY) Mar 08 04:51:34 PM:   Affine conic cons.     : 0               
(CVXPY) Mar 08 04:51:34 PM:   Disjunctive cons.      : 0             

(CVXPY) Mar 08 04:54:10 PM: 12031    12019    8462     69       3.9532099828e+01     2.0509455347e+01     48.12       155.9 
(CVXPY) Mar 08 04:54:13 PM: 12324    12312    8611     93       3.9532099828e+01     2.0509455347e+01     48.12       159.7 
(CVXPY) Mar 08 04:54:18 PM: 12595    12583    8738     115      3.9532099828e+01     2.0509455347e+01     48.12       164.0 
(CVXPY) Mar 08 04:54:22 PM: 12885    12873    8874     139      3.9532099828e+01     2.0509455347e+01     48.12       168.3 
(CVXPY) Mar 08 04:54:26 PM: 13173    13161    9028     163      3.9532099828e+01     2.0509455347e+01     48.12       172.0 
(CVXPY) Mar 08 04:54:29 PM: 13465    13453    9144     187      3.9532099828e+01     2.0509455347e+01     48.12       175.6 
(CVXPY) Mar 08 04:54:33 PM: 13755    13743    9268     211      3.9532099828e+01     2.0509455347e+01     48.12       179.2 
(CVXPY) Mar 08 04:54:36 PM: 14048    14036    9377     235      3.9532099828e+01     2.0509455347e+01     48.12       182.7 


(CVXPY) Mar 08 04:57:58 PM: 33578    32431    8073     30       3.7016812320e+01     2.6086788251e+01     29.53       383.8 
(CVXPY) Mar 08 04:58:01 PM: 33906    32749    8019     31       3.7016812320e+01     2.6086788310e+01     29.53       387.2 
(CVXPY) Mar 08 04:58:04 PM: 34235    33065    7990     37       3.7016812320e+01     2.6279049628e+01     29.01       390.6 
(CVXPY) Mar 08 04:58:08 PM: 34582    33383    7933     33       3.7016812320e+01     2.6427492832e+01     28.61       394.0 
(CVXPY) Mar 08 04:58:11 PM: 34997    33701    7772     37       3.6962095894e+01     2.6657983184e+01     27.88       397.5 
(CVXPY) Mar 08 04:58:15 PM: 35492    34003    7525     29       3.6962095894e+01     2.6826306838e+01     27.42       401.0 
(CVXPY) Mar 08 04:58:18 PM: 36023    34314    7286     33       3.6962095894e+01     2.6972677368e+01     27.03       404.5 
(CVXPY) Mar 08 04:58:22 PM: 36593    34627    6992     31       3.6962095894e+01     2.6975832112e+01     27.02       408.0 


...

-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Mar 08 09:11:09 PM: Invoking solver MOSEK  to obtain a solution.


(CVXPY) Mar 08 09:11:09 PM: Problem
(CVXPY) Mar 08 09:11:09 PM:   Name                   :                 
(CVXPY) Mar 08 09:11:09 PM:   Objective sense        : minimize        
(CVXPY) Mar 08 09:11:09 PM:   Type                   : CONIC (conic optimization problem)
(CVXPY) Mar 08 09:11:09 PM:   Constraints            : 4013            
(CVXPY) Mar 08 09:11:09 PM:   Affine conic cons.     : 0               
(CVXPY) Mar 08 09:11:09 PM:   Disjunctive cons.      : 0               
(CVXPY) Mar 08 09:11:09 PM:   Cones                  : 1               
(CVXPY) Mar 08 09:11:09 PM:   Scalar variables       : 2010            
(CVXPY) Mar 08 09:11:09 PM:   Matrix variables       : 

(CVXPY) Mar 08 09:14:17 PM: 13217    13205    13170    412      8.8790819165e+04     -5.8397346664e-06    100.00      187.8 
(CVXPY) Mar 08 09:14:21 PM: 13526    13514    13479    438      8.8790819165e+04     -5.8397346664e-06    100.00      191.4 
(CVXPY) Mar 08 09:14:25 PM: 13833    13821    13786    464      8.8790819165e+04     -5.8397346664e-06    100.00      195.2 
(CVXPY) Mar 08 09:14:28 PM: 14135    14123    14088    489      8.8790819165e+04     -5.8397346664e-06    100.00      198.6 
(CVXPY) Mar 08 09:14:31 PM: 14443    14426    14376    174      3.7105291898e+01     -5.8397346664e-06    100.00      202.2 
(CVXPY) Mar 08 09:14:36 PM: 14733    14702    14638    110      3.7105291898e+01     -5.8397346664e-06    100.00      206.6 
(CVXPY) Mar 08 09:14:40 PM: 15023    14992    14928    111      3.7105291898e+01     -5.8397346664e-06    100.00      211.1 
(CVXPY) Mar 08 09:14:45 PM: 15311    15280    15216    107      3.7105291898e+01     -5.8397346664e-06    100.00      215.4 


(CVXPY) Mar 08 09:18:57 PM: 32988    32937    30473    74       3.4550991690e+01     -5.8397346664e-06    100.00      467.9 
(CVXPY) Mar 08 09:19:02 PM: 33282    33231    30767    114      3.4550991690e+01     -5.8397346664e-06    100.00      472.6 
(CVXPY) Mar 08 09:19:07 PM: 33576    33525    31061    113      3.4550991690e+01     -5.8397346664e-06    100.00      477.3 
(CVXPY) Mar 08 09:19:12 PM: 33846    33795    31331    27       3.4550991690e+01     -5.8397346664e-06    100.00      482.2 
(CVXPY) Mar 08 09:19:16 PM: 34139    34088    31624    106      3.4550991690e+01     -5.8397346664e-06    100.00      486.6 
(CVXPY) Mar 08 09:19:20 PM: 34433    34382    31918    83       3.4550991690e+01     -5.8397346664e-06    100.00      491.0 
(CVXPY) Mar 08 09:19:25 PM: 34727    34676    32212    104      3.4550991690e+01     -5.8397346664e-06    100.00      495.4 
(CVXPY) Mar 08 09:19:29 PM: 35021    34970    32502    102      3.4550991690e+01     -5.8397346664e-06    100.00      499.6 


(CVXPY) Mar 08 09:22:28 PM: Solver (including time spent in interface) took 6.785e+02 seconds
final target accuracy:: inf
total running time: 721.8514568805695 [s]
cost:: 34.23600313504944
 
tau = 0.25
 
skip_presolve: False
 
pre-pass oracle...
LB = 0.0 | self.UB = 4.852386587771204 | pre-pruned: 103
 
colinearity detected...
colinearity detected...
 
empirical cstr (LB): 0.6660000000000001
current lower-bound: 1.6637207807357703
 
 
empirical cstr (UB): 0.7500000000000002
current upper-bound: 4.852386587771204
 
 
trial @1 | remaining checks [%] -> 98.43
trial @41 | remaining checks [%] -> 89.06
trial @97 | remaining checks [%] -> 79.68
pruned! @113
pruned! @121
trial @127 | remaining checks [%] -> 70.31
trial @146 | remaining checks [%] -> 60.93
trial @214 | remaining checks [%] -> 51.56
trial @317 | remaining checks [%] -> 42.18
trial @365 | remaining checks [%] -> 32.81
trial @404 | remaining checks [%] -> 23.43
trial @445 | remaining checks [%] -> 14.06
trial @477 | remaining che

(CVXPY) Mar 08 09:26:42 PM: 6064     6047     5243     18       7.4947788079e+00     3.8626612331e+00     48.46       70.1  
(CVXPY) Mar 08 09:26:46 PM: 6318     6301     5355     39       7.4947788079e+00     3.8626612331e+00     48.46       74.1  
(CVXPY) Mar 08 09:26:50 PM: 6572     6555     5551     50       7.4947788079e+00     3.8626612331e+00     48.46       78.3  
(CVXPY) Mar 08 09:26:55 PM: 6828     6811     5751     20       7.4947788079e+00     3.8626612331e+00     48.46       82.6  
(CVXPY) Mar 08 09:26:59 PM: 7083     7066     5946     63       7.4947788079e+00     3.8935530871e+00     48.05       86.6  
(CVXPY) Mar 08 09:27:02 PM: 7337     7320     6156     83       7.4947788079e+00     3.9071134926e+00     47.87       90.4  
(CVXPY) Mar 08 09:27:06 PM: 7591     7574     6360     52       7.4947788079e+00     3.9071134926e+00     47.87       94.1  
(CVXPY) Mar 08 09:27:10 PM: 7848     7831     6557     20       7.4947788079e+00     3.9081737237e+00     47.85       97.6  
